In [1]:
# importing libraries 
import speech_recognition as sr 

import os
import shutil
import sys

from pydub import AudioSegment
from pydub.silence import split_on_silence

import pydub

The error:
> (WindowsError: [Error 2] The system can not find the file specified)

was solved copying the ffmpeg files (ffmpeg.exe, ffplay.exe, ffprobe.exe) to the pydub module folder (C:\Anaconda3\envs\datascience\Lib\site-packages\pydub)

Some blogs says that you can add the following lines to the code:
> sys.path.append(r'C:\Anaconda3\ffmpeg\bin')<br>
> AudioSegment.converter = r"C:\Anaconda3\ffmpeg\bin\ffmpeg.exe"<br>
> AudioSegment.ffprobe   = r"C:\Anaconda3\ffmpeg\bin\ffprobe.exe"

These don't work

In [2]:
# a function that splits the audio file into chunks and applies speech recognition
def get_large_audio_transcription(file, language, energy_threshold=300, duration=None, offset=None, show_all=None, noise=0):
    """
    Splitting the large audio file into chunks and apply speech recognition on each of these chunks
    """
    
    # create a speech recognition object
    recognizer = sr.Recognizer()

    # open the audio file using pydub
    if '.wav' in file:
        audio_file = AudioSegment.from_wav(file)
    elif '.mp3' in file:
        audio_file = AudioSegment.from_file(file, "mp3")
    elif '.ogg' in file:
        audio_file = AudioSegment.from_ogg(file)
    elif '.flv' in file:
        audio_file = AudioSegment.from_flv(file)
    elif '.mp4' in file:
        audio_file = AudioSegment.from_file(file, "mp4")
    elif '.wma' in file:
        audio_file = AudioSegment.from_file(file, "wma")
    elif '.aiff' in file:
        audio_file = AudioSegment.from_file(file, "aac")
    else:
        audio_file = AudioSegment.from_file(file)  
    
    # split audio_file where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(audio_file,
                              min_silence_len = 500, # experiment with this value for your target audio file
                              silence_thresh = audio_file.dBFS-14, # adjust this per requirement
                              keep_silence=500, # keep the silence for 1 second, adjustable as well
                             )
    folder_name = "audio-chunks-temp"
    
    # create a directory to store the audio chunks
    if os.path.isdir(folder_name):
        shutil.rmtree(folder_name)
    os.mkdir(folder_name)
    
    whole_text = ""
    with open(file.split('.')[0]+'.txt', 'w') as f:
        # process each chunk 
        print("Let's begin:")
        for i, audio_chunk in enumerate(chunks, start=1):
            # export audio chunk and save it in the `folder_name` directory.
            chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
            
            audio_chunk.export(chunk_filename, format="wav")
            
            # recognize the chunk
            with sr.AudioFile(chunk_filename) as source:
                # Adjust for ambient noise and record
                if noise>0:
                    recognizer.adjust_for_ambient_noise(source, duration=noise)
                
                # Record the audio
                audio_listened = recognizer.record(source,
                                                   duration=duration, #Listen from the begining to duration value.
                                                   offset=offset) #used to skip over a specific seconds at the start.
                
                # Set the energy threshold
                recognizer.energy_threshold = energy_threshold
    
                # try converting it to text
                try:
                    text = recognizer.recognize_google(audio_listened, language=language)
                except sr.UnknownValueError as e:
                    msg_to_write = f"Error: {str(e)}\n" 
                    print(msg_to_write)
                else:
                    text = f"{text.capitalize()}. "
                    msg_to_write = f"{i}: {text}\n"
                    print(chunk_filename, ":", text)
                    whole_text += text
                
                # Write the text to the file
                f.write(msg_to_write)
        print('End.')
    
    # return the text for all chunks detected
    return whole_text

In [3]:
path = "7601-291468-0006.wav"
print("\nFull text:", get_large_audio_transcription(path, language='en-US'))

Let's begin:
audio-chunks-temp\chunk1.wav : His abode which you had fixed in a bowery or country seat. 
audio-chunks-temp\chunk2.wav : Have a short distance from the city. 
audio-chunks-temp\chunk3.wav : Just at what is now called dutch street. 
audio-chunks-temp\chunk4.wav : Sooner bounded with proofs of his ingenuity. 
audio-chunks-temp\chunk5.wav : Patent smokejack. 
audio-chunks-temp\chunk6.wav : It required a horse to work some. 
audio-chunks-temp\chunk7.wav : Dutch oven roasted meat without fire. 
audio-chunks-temp\chunk8.wav : Carts that went before the horses. 
audio-chunks-temp\chunk9.wav : Weather cox that turned against the wind and other wrongheaded contrivances. 
audio-chunks-temp\chunk10.wav : So just understand can found it all beholders. 
End.

Full text: His abode which you had fixed in a bowery or country seat. Have a short distance from the city. Just at what is now called dutch street. Sooner bounded with proofs of his ingenuity. Patent smokejack. It required a hors

In [4]:
with open('7601-291468-0006.txt', 'r') as f:
    print(f.read())

1: His abode which you had fixed in a bowery or country seat. 
2: Have a short distance from the city. 
3: Just at what is now called dutch street. 
4: Sooner bounded with proofs of his ingenuity. 
5: Patent smokejack. 
6: It required a horse to work some. 
7: Dutch oven roasted meat without fire. 
8: Carts that went before the horses. 
9: Weather cox that turned against the wind and other wrongheaded contrivances. 
10: So just understand can found it all beholders. 



In [5]:
path = "multiple-speakers-16k.wav"
#print("\nFull text:", get_large_audio_transcription(path, language='en-US'))
_ = get_large_audio_transcription(path, language='en-US')

Let's begin:
audio-chunks-temp\chunk1.wav : One of the limitations of the speech recognition lottery. 
audio-chunks-temp\chunk2.wav : Is that it doesn't recognize different speakers invoices. 
audio-chunks-temp\chunk3.wav : It will just return it all as one block a text. 
End.


In [6]:
with open('multiple-speakers-16k.txt', 'r') as f:
    print(f.read())

1: One of the limitations of the speech recognition lottery. 
2: Is that it doesn't recognize different speakers invoices. 
3: It will just return it all as one block a text. 



In [7]:
path = "AUD-20180918-WA0000.mp3"
#print("\nFull text:", get_large_audio_transcription(path, language='es-US'))
_ = get_large_audio_transcription(path, language='es-US')

Let's begin:
audio-chunks-temp\chunk1.wav : Sabe usted dónde está el secreto de los norteamericanos. 
audio-chunks-temp\chunk2.wav : Muy sencillo. 
audio-chunks-temp\chunk3.wav : Hace más de 150 años aprendieron algo que en latinoamérica pareciera que no hemos ni queremos aprender. 
audio-chunks-temp\chunk4.wav : Son sólo diez muy simples premisas. 
audio-chunks-temp\chunk5.wav : Es el decálogo de abraham lincoln. 
audio-chunks-temp\chunk6.wav : Usted no puede crear prosperidad desalentando la iniciativa propia. 
audio-chunks-temp\chunk7.wav : Usted no puede fortalecer al débil debilitando al fuerte. 
audio-chunks-temp\chunk8.wav : Usted no puede ayudar a los pequeños aplastando a los grandes. 
audio-chunks-temp\chunk9.wav : Usted no puede ayudar al pobre destruyendo al rico. 
audio-chunks-temp\chunk10.wav : Usted no puede elevar al asalariado presionando a quién paga el salario. 
audio-chunks-temp\chunk11.wav : Usted no puede resolver sus problemas mientras gaste más de lo que gana. 


In [8]:
with open('AUD-20180918-WA0000.txt', 'r') as f:
    print(f.read())

1: Sabe usted dónde está el secreto de los norteamericanos. 
2: Muy sencillo. 
3: Hace más de 150 años aprendieron algo que en latinoamérica pareciera que no hemos ni queremos aprender. 
4: Son sólo diez muy simples premisas. 
5: Es el decálogo de abraham lincoln. 
6: Usted no puede crear prosperidad desalentando la iniciativa propia. 
7: Usted no puede fortalecer al débil debilitando al fuerte. 
8: Usted no puede ayudar a los pequeños aplastando a los grandes. 
9: Usted no puede ayudar al pobre destruyendo al rico. 
10: Usted no puede elevar al asalariado presionando a quién paga el salario. 
11: Usted no puede resolver sus problemas mientras gaste más de lo que gana. 
12: Usted no puede promover la fraternidad de la humanidad. 
13: Admitiendo e incitando el odio de clases. 
14: Usted no puede garantizar una adecuada seguridad con dinero prestado. 
15: Usted no puede formar el carácter y el valor del hombre. 
16: Quitándole su independencia. 
17: Iniciativa. 
18: Usted no puede ayudar

In [9]:
path = "CocaCola.mp3"
#print("\nFull text:", get_large_audio_transcription(path, language='es-US'))
_ = get_large_audio_transcription(path, language='es-US')

Let's begin:
Error: 

Error: 

Error: 

Error: 

Error: 

Error: 

audio-chunks-temp\chunk7.wav : El objetivo que nosotros tenemos el del departamento de recursos humanos es conseguir. 
audio-chunks-temp\chunk8.wav : La gente que trabaja en cocacola. 
audio-chunks-temp\chunk9.wav : Este agusto tenga los medios adecuados para hacer su trabajo. 
audio-chunks-temp\chunk10.wav : Y que por tanto por ese abajo otros tenemos una iniciativa que es el instituto de la felicidad en el que trabajamos en muchas acciones hacia fuera pero un poco la filosofía de la compañía es que lo mismo que nosotros mostramos aquí afuera o declinamos hacia dentro. 
audio-chunks-temp\chunk11.wav : Y básicamente lo que intentamos es las mismas acciones. 
audio-chunks-temp\chunk12.wav : Tenemos aquí afuera. 
audio-chunks-temp\chunk13.wav : Hacerlas con nuestros empleados. 
audio-chunks-temp\chunk14.wav : Y y perseguir la máxima del instituto que es que la gente que está feliz. 
audio-chunks-temp\chunk15.wav : Pues tr

In [10]:
with open('CocaCola.txt', 'r') as f:
    print(f.read())

Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
7: El objetivo que nosotros tenemos el del departamento de recursos humanos es conseguir. 
8: La gente que trabaja en cocacola. 
9: Este agusto tenga los medios adecuados para hacer su trabajo. 
10: Y que por tanto por ese abajo otros tenemos una iniciativa que es el instituto de la felicidad en el que trabajamos en muchas acciones hacia fuera pero un poco la filosofía de la compañía es que lo mismo que nosotros mostramos aquí afuera o declinamos hacia dentro. 
11: Y básicamente lo que intentamos es las mismas acciones. 
12: Tenemos aquí afuera. 
13: Hacerlas con nuestros empleados. 
14: Y y perseguir la máxima del instituto que es que la gente que está feliz. 
15: Pues trabajando nosotros lo definimos como una universidad corporativa virtual. 
16: Al final la filosofía de la universidad sedes recoger las mejores prácticas a nivel mundial. 
17: Cocacola está presente en más de 206 países. 
18: Y acercar las mejores prácticas a los emplead